# Load the model in python and transform to onnx

### https://onnxruntime.ai/docs/tutorials/csharp/bert-nlp-csharp-console-app.html

##### opset_version compatibility

https://onnxruntime.ai/docs/reference/compatibility.html#:~:text=ONNX%20Runtime%20supports%20all%20opsets%20from%20the%20latest,with%20ONNX%20opset%20versions%20in%20the%20range%20%5B7-9%5D.

- install onnx , torch, transformers

### TORCH
https://pytorch.org/tutorials//beginner/onnx/export_simple_model_to_onnx_tutorial.html

###  Class c# to work with BERTTokenizer Models

https://github.com/NMZivkovic/BertTokenizers

# Model
- bert-large-uncased-whole-word-masking-finetuned-squad

### Suported models


- BERT Base
- BERT Large
- BERT German
- BERT Multilingual
- BERT Base Uncased
- BERT Large Uncased

# Netron
https://github.com/lutzroeder/netron/releases/latest?tab=readme-ov-file

In [32]:
import torch
from transformers import BertForQuestionAnswering

model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model_path = "./models/" + model_name + "-17b.onnx"
model = BertForQuestionAnswering.from_pretrained(model_name)


In [33]:
torch.cuda.is_available()

False

In [34]:
torch.__version__

'2.2.2+cpu'

In [35]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [36]:
import onnx

In [37]:
onnx.__version__

'1.16.1'

In [38]:
# Generate dummy inputs to the model. Adjust if neccessary.
inputs = {
        # list of numerical ids for the tokenized text
        'input_ids':   torch.randint(32, [1, 32], dtype=torch.long), 
        # dummy list of ones
        'attention_mask': torch.ones([1, 32], dtype=torch.long),     
        # dummy list of ones
        'token_type_ids':  torch.ones([1, 32], dtype=torch.long)     
    }

symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
torch.onnx.export(model,                                         
# model being run
                  (inputs['input_ids'],
                   inputs['attention_mask'], 
                   inputs['token_type_ids']),                    # model input (or a tuple for multiple inputs)
                  model_path,                                    # where to save the model (can be a file or file-like object)
                  opset_version=17,  # the ONNX version to export the model to MAX 17
                  verbose= True ,
                  do_constant_folding=True,                      # whether to execute constant folding for optimization
                  input_names=['input_ids',
                               'input_mask', 
                               'segment_ids'],                   # the model's input names
                  output_names=['start_logits', "end_logits"],   # the model's output names
                  dynamic_axes={'input_ids': symbolic_names,
                                'input_mask' : symbolic_names,
                                'segment_ids' : symbolic_names,
                                'start_logits' : symbolic_names, 
                                'end_logits': symbolic_names})   # variable length axes/dynamic input By default the exported model will have the shapes of all input and output tensors
                                                                 #set to exactly match those given in ``args``. To specify axes of tensors as
                                                                # dynamic (i.e. known only at run-time), set ``dynamic_axes`` to a dict with schema